In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import stan # python3 -m pip install pystan
import nest_asyncio
nest_asyncio.apply()
import warnings

In [17]:
df = pd.read_csv("Number_Pivoted.csv")
df = df.set_index("Year")

In [18]:
data = dict(N=len(df), x=list(range(1, 23)), y=df["[All]"].to_numpy())

In [20]:
file_path = "stan/linear.stan"

# Read the Stan code from the file
with open(file_path, "r") as file:
    stan_code = file.read()
print(stan_code)

warnings.filterwarnings("ignore")

posterior = stan.build(stan_code, data=data)

data {
  int<lower=0> N;
  vector[N] x;
  vector[N] y;
}
parameters {
  real alpha;
  real beta;
  real<lower=0> sigma;
}
model {
  // priors
  alpha ~ normal(42000, 1000);
  beta ~ normal(0, 100);
  sigma ~ cauchy(0, 2.5); 
  
  y ~ normal(alpha + beta * x, sigma);
}
Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_kw02y3tr/model_sww5sjly.stan', line 14, column 19: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_kw02y3tr/model_sww5sjly.stan', line 13, column 24: Argument
    1000 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_kw02y3tr/model_sww5sjly.stan', line 13, column 17: Argument
    42000 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).


In [21]:
fit = posterior.sample(num_chains=4, num_samples=1000)
linear_df = fit.to_frame()  # pandas `DataFrame, requires pandas

Sampling:   0%
Sampling:   1% (100/8000)
Sampling:  26% (2100/8000)
Sampling:  51% (4100/8000)
Sampling:  76% (6100/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 5.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.52 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.38 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.55 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.62 seconds.
  Adjust your expectations accordingly!


In [22]:
linear_df

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,alpha,beta,sigma
draws,,,,,,,,,,
0,-200.024493,0.997037,0.480220,3.0,7.0,0.0,201.313813,41336.446038,-36.510688,4469.837638
1,-202.127573,0.927432,0.485257,2.0,3.0,0.0,202.337088,40228.728406,-110.768375,3511.142038
2,-199.953228,0.935940,0.619661,2.0,7.0,0.0,200.674962,41028.933376,-14.310125,4066.680136
3,-199.686192,0.931058,0.639573,2.0,7.0,0.0,202.529059,42475.116550,-184.110450,3681.445970
4,-200.309995,0.992017,0.480220,2.0,3.0,0.0,200.478237,41163.186326,-16.866200,4474.501453
...,...,...,...,...,...,...,...,...,...,...
3995,-199.685471,0.725684,0.639573,3.0,7.0,0.0,202.729103,41853.934173,-174.464113,3431.302790
3996,-205.575991,0.618408,0.480220,2.0,7.0,0.0,207.314288,40298.036830,-200.696547,3799.879581
3997,-199.889208,1.000000,0.485257,3.0,7.0,0.0,200.606629,41136.938782,-129.676267,4038.510844


In [92]:
df.head()

,[0],[1-4],[10-14],[15-19],[20-24],[25-29],[30-34],[35-39],[40-44],[45-49],[5-9],[50-54],[55-59],[60-64],[65-69],[70-74],[75-79],[80-84],[85+],[All]
Year,,,,,,,,,,,,,,,,,,,,
2000,163.0,567.0,936.0,5176.0,5264.0,3679.0,3153.0,3549.0,3392.0,2938.0,751.0,2413.0,1890.0,1581.0,1414.0,1583.0,1724.0,1390.0,1276.0,42866.0
2001,140.0,564.0,899.0,5155.0,5465.0,3675.0,3196.0,3522.0,3549.0,3047.0,669.0,2557.0,1915.0,1518.0,1393.0,1571.0,1792.0,1459.0,1209.0,43328.0
2002,122.0,541.0,900.0,5549.0,5760.0,3689.0,3343.0,3441.0,3616.0,3270.0,636.0,2700.0,2005.0,1755.0,1456.0,1615.0,1734.0,1506.0,1264.0,44937.0
2003,146.0,510.0,929.0,5265.0,5678.0,3649.0,3195.0,3293.0,3696.0,3302.0,608.0,2798.0,2252.0,1741.0,1430.0,1533.0,1655.0,1521.0,1337.0,44555.0
2004,140.0,524.0,941.0,5180.0,5726.0,3825.0,3162.0,3195.0,3470.0,3435.0,601.0,2867.0,2287.0,1789.0,1549.0,1414.0,1650.0,1482.0,1242.0,44505.0


In [23]:
df_sep = df.drop("[All]", axis=1).T
df_sep.shape

(19, 22)

In [24]:
data_sep = dict(
    N = 19,
    Y = 22,
    accidentData = df_sep.values,
    xpred = 2000)

In [ ]:
file_path = "stan/separate.stan"

# Read the Stan code from the file
with open(file_path, "r") as file:
    stan_code = file.read()

warnings.filterwarnings("ignore")

posterior = stan.build(stan_code, data=data_sep)

In [27]:
fit = posterior.sample(num_chains=4, num_samples=1000)
separate_df = fit.to_frame()  # pandas `DataFrame, requires pandas

Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (401/8000)
Sampling:   8% (601/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1500/8000)
Sampling:  20% (1600/8000)
Sampling:  22% (1800/8000)
Sampling:  24% (1900/8000)
Sampling:  25% (2000/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2300/8000)
Sampling:  31% (2500/8000)
Sampling:  34% (2700/8000)
Sampling:  36% (2900/8000)
Sampling:  39% (3100/8000)
Sampling:  54% (4300/8000)
Sampling:  70% (5600/8000)
Sampling:  84% (6700/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 0.000482 seconds
  1000 transitions using 10 leapfrog steps per transition would take 4.82 seconds

In [28]:
separate_df.describe()

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,alpha.1,alpha.2,alpha.3,...,pred.10,pred.11,pred.12,pred.13,pred.14,pred.15,pred.16,pred.17,pred.18,pred.19
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.0,4000.000000,4000.000000,4000.000000,4000.000000,...,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,-2951.273965,0.897869,0.165139,4.946250,30.556000,0.0,2979.665048,147.698037,543.460366,670.399519,...,396.482367,595.086500,426.386622,368.629170,397.633386,604.203964,675.478238,606.199777,685.884438,997.168827
std,5.544604,0.124540,0.010851,0.225552,2.770701,0.0,7.789628,9.209099,19.460001,86.533317,...,2625.870295,80.858851,2572.191937,2301.738041,1729.596473,987.798956,661.512022,755.325913,497.156189,203.841797
min,-2980.241500,0.006091,0.153262,4.000000,15.000000,0.0,2956.971017,108.714746,468.485089,347.525569,...,-10924.647247,233.632819,-8606.801423,-10168.004102,-9008.725339,-4758.073494,-2759.372970,-2831.341745,-2290.260508,-433.463198
25%,-2954.721773,0.867323,0.158446,5.000000,31.000000,0.0,2974.344664,141.761964,530.381621,613.618325,...,-1325.940860,544.528748,-1197.317464,-1100.214274,-696.215390,52.066256,295.231411,148.212684,383.254126,897.799348
50%,-2950.945736,0.937286,0.162349,5.000000,31.000000,0.0,2979.257141,147.626030,543.802748,677.547405,...,463.621146,597.800023,370.738860,394.556409,405.477818,692.682268,712.530972,617.214071,699.984411,1026.570711
75%,-2947.334955,0.978042,0.169042,5.000000,31.000000,0.0,2984.553006,153.664614,556.685859,733.857579,...,2132.065445,647.647333,2077.023958,1864.662344,1498.184566,1189.753302,1077.879610,1074.982270,1011.355362,1130.671876
max,-2934.882040,1.000000,0.182595,5.000000,63.000000,0.0,3016.319632,184.223566,616.364399,913.629358,...,11641.215488,1011.406858,9375.240121,10310.946473,7969.310116,6602.648861,4680.611278,3527.226574,2606.576512,1762.057514


In [30]:
file_path = "stan/pooled.stan"

# Read the Stan code from the file
with open(file_path, "r") as file:
    stan_code = file.read()

warnings.filterwarnings("ignore")

posterior = stan.build(stan_code, data=data_sep)

data {
  int<lower=0> N; // age group
  int<lower=0> Y; // year
  matrix[N,Y] accidentData;//accident data
  int xpred;
}


parameters {
  real alpha;
  real beta;
  real<lower=0> sigma;
}

transformed parameters{
  vector[Y]mu;
  //linear model
  for(j in 1:Y)
    mu[j]=alpha+beta*j;
}


model {

    alpha~normal(0,100);
    beta~normal(0,10);

  for(j in 1:Y){
    accidentData[,j]~normal(mu[j],sigma);
  }
}

generated quantities{
  //log likelihood
  matrix[N,Y] log_lik;
  matrix[N,Y] yrep;
  //accident prediction in 2020 in different police force
  vector[N] pred;
  for(i in 1:N){
    // 2005 -> 1, 2006 -> 2, ..., 2020 -> 16 
    pred[i]=normal_rng(alpha+beta*(xpred-2004),sigma);
  }
  
  for(i in 1:N){
    for(j in 1:Y){
      // do posterior sampling and try to reproduce the original data
      yrep[i,j]=normal_rng(mu[j],sigma);
      // prepare log likelihood for PSIS-LOO 
      log_lik[i,j]=normal_lpdf(accidentData[i,j]|mu[j],sigma);
    }
  }
  
}
Building...

In file included from /home/mantyke1/.cache/httpstan/4.6.1/models/addrqjqf/model_addrqjqf.cpp:2:
In file included from /opt/software/lib/python3.10/site-packages/httpstan/include/stan/model/model_header.hpp:4:
In file included from /opt/software/lib/python3.10/site-packages/httpstan/include/stan/math.hpp:19:
In file included from /opt/software/lib/python3.10/site-packages/httpstan/include/stan/math/rev.hpp:8:
In file included from /opt/software/lib/python3.10/site-packages/httpstan/include/stan/math/rev/core.hpp:28:
In file included from /opt/software/lib/python3.10/site-packages/httpstan/include/stan/math/rev/core/operator_addition.hpp:6:
/opt/software/lib/python3.10/site-packages/httpstan/include/stan/math/prim/err/check_matching_dims.hpp:56:23: warning: comparison of integers of different signs: 'int' and 'std::vector::size_type' (aka 'unsigned long') [-Wsign-compare]
    for (int i = 0; i < y1_d.size(); i++) {
                    ~ ^ ~~~~~~~~~~~
In file included from /home/mantyke1


Building: 33.5s, done.Messages from stanc:
Warning in '/tmp/httpstan_e2jworpd/model_addrqjqf.stan', line 25, column 19: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.


In [32]:
fit_pooled = posterior.sample(num_chains=4, num_samples=1000)
pooled_df = fit.to_frame()  # pandas `DataFrame, requires pandas

Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:  25% (2003/8000)
Sampling:  50% (4002/8000)
Sampling:  75% (6001/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 3.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.34 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 8.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.88 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.58 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.31 seconds.
  Adjust your expectations accordingly!


In [33]:
pooled_df.describe()

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,alpha.1,alpha.2,alpha.3,...,pred.10,pred.11,pred.12,pred.13,pred.14,pred.15,pred.16,pred.17,pred.18,pred.19
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.0,4000.000000,4000.000000,4000.000000,4000.000000,...,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,-2951.273965,0.897869,0.165139,4.946250,30.556000,0.0,2979.665048,147.698037,543.460366,670.399519,...,396.482367,595.086500,426.386622,368.629170,397.633386,604.203964,675.478238,606.199777,685.884438,997.168827
std,5.544604,0.124540,0.010851,0.225552,2.770701,0.0,7.789628,9.209099,19.460001,86.533317,...,2625.870295,80.858851,2572.191937,2301.738041,1729.596473,987.798956,661.512022,755.325913,497.156189,203.841797
min,-2980.241500,0.006091,0.153262,4.000000,15.000000,0.0,2956.971017,108.714746,468.485089,347.525569,...,-10924.647247,233.632819,-8606.801423,-10168.004102,-9008.725339,-4758.073494,-2759.372970,-2831.341745,-2290.260508,-433.463198
25%,-2954.721773,0.867323,0.158446,5.000000,31.000000,0.0,2974.344664,141.761964,530.381621,613.618325,...,-1325.940860,544.528748,-1197.317464,-1100.214274,-696.215390,52.066256,295.231411,148.212684,383.254126,897.799348
50%,-2950.945736,0.937286,0.162349,5.000000,31.000000,0.0,2979.257141,147.626030,543.802748,677.547405,...,463.621146,597.800023,370.738860,394.556409,405.477818,692.682268,712.530972,617.214071,699.984411,1026.570711
75%,-2947.334955,0.978042,0.169042,5.000000,31.000000,0.0,2984.553006,153.664614,556.685859,733.857579,...,2132.065445,647.647333,2077.023958,1864.662344,1498.184566,1189.753302,1077.879610,1074.982270,1011.355362,1130.671876
max,-2934.882040,1.000000,0.182595,5.000000,63.000000,0.0,3016.319632,184.223566,616.364399,913.629358,...,11641.215488,1011.406858,9375.240121,10310.946473,7969.310116,6602.648861,4680.611278,3527.226574,2606.576512,1762.057514
